In [1]:
!pip install seaborn
!pip install skicit-learn


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement skicit-learn (from versions: none)
ERROR: No matching distribution found for skicit-learn

[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [87]:
import pandas as pd
import numpy as np
import seaborn as sns
import graphviz
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import cross_val_score,StratifiedKFold,train_test_split,cross_validate,GridSearchCV

Load Dataset

In [152]:
heart_data = pd.read_csv('heart.csv')

In [153]:
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [154]:
heart_data.shape

(918, 12)

In [155]:
heart_data['HeartDisease'].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

In [156]:
heart_data.columns

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')

In [157]:
heart_data.drop(['RestingECG', 'ExerciseAngina','Oldpeak','ST_Slope'], axis = 1, inplace=True)

In [158]:
heart_data.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,HeartDisease
0,40,M,ATA,140,289,0,172,0
1,49,F,NAP,160,180,0,156,1
2,37,M,ATA,130,283,0,98,0
3,48,F,ASY,138,214,0,108,1
4,54,M,NAP,150,195,0,122,0


In [159]:
# kolom_label_encode = heart_data[['Sex', 'ChestPainType']]

# for kolom in kolom_label_encode:
label_encoder = LabelEncoder()

heart_data[['Sex', 'ChestPainType']]=heart_data[['Sex', 'ChestPainType']].apply(label_encoder.fit_transform)
heart_data_tree = heart_data 

heart_data_tree.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,MaxHR,HeartDisease
0,40,1,1,140,289,0,172,0
1,49,0,2,160,180,0,156,1
2,37,1,1,130,283,0,98,0
3,48,0,0,138,214,0,108,1
4,54,1,2,150,195,0,122,0


In [160]:
# memisahkan data dan label
X = heart_data_tree.drop(columns='HeartDisease', axis=1)
Y = heart_data_tree['HeartDisease']

Split Data

In [161]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3)

In [162]:
print(X_train.shape, X_test.shape)

(642, 7) (276, 7)


Membuat Model

In [163]:
kf = StratifiedKFold(n_splits=5)
X_tree = DecisionTreeClassifier()
X_score=cross_validate(X_tree,X=X_train,y=Y_train,cv=kf,scoring=['accuracy','recall','precision','roc_auc'])
X_score

{'fit_time': array([0.00694895, 0.00697756, 0.00903749, 0.0119648 , 0.01133227]),
 'score_time': array([0.02393675, 0.02094579, 0.01789451, 0.02493238, 0.02146339]),
 'test_accuracy': array([0.73643411, 0.73643411, 0.7421875 , 0.71875   , 0.7265625 ]),
 'test_recall': array([0.7826087 , 0.71014493, 0.73913043, 0.71014493, 0.72463768]),
 'test_precision': array([0.73972603, 0.77777778, 0.77272727, 0.75384615, 0.75757576]),
 'test_roc_auc': array([0.73297101, 0.7384058 , 0.74244657, 0.71947924, 0.72672562])}

In [164]:
X_tree = X_tree.fit(X_train,Y_train)

Membuat model evaluasi untuk mengukur tingkat akurasi

In [165]:
X_train_prediction = X_tree.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [166]:
print(training_data_accuracy)

1.0


In [167]:
X_test_prediction = X_tree.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [168]:
print(test_data_accuracy)

0.7427536231884058


Model Prediksi

In [198]:
input_data = (48, 'F', 'ASY', 138, 214, 0, 108)

arr1 = np.array(input_data)
arr1 = arr1.reshape(1,-1)

# Indeks-indeks yang ingin dilakukan encoding
indeks_encode = [1, 2]

# Melakukan encoding pada indeks-indeks yang dipilih
for indeks in indeks_encode:
    label_encoder = LabelEncoder()
    arr1[:, indeks] = label_encoder.fit_transform(arr1[:, indeks])

prediction = X_tree.predict(arr1)
print(prediction)

if (prediction[0] == 0):
    print("Anda tidak terindikasi gagal jantung")
else:
    print("Anda terindikasi gagal jantung")

[1]
Anda terindikasi gagal jantung


c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


Simpan Model

In [199]:
import pickle

In [200]:
filename = 'heartFailure.sav'
pickle.dump(X_tree, open(filename, 'wb'))